In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np

In [14]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.thing = nn.Parameter(torch.rand(10), requires_grad=True)
    
#     def forward(self):
#         return self.thing

In [15]:
# net = Net()

# learning_rate = 0.1

# loss_fn = nn.MSELoss()

# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [16]:
# ideal_weights = torch.tensor([0.0027] * 10)
# for epoch in range(100):
#     optimizer.zero_grad()
#     pred = net()
#     print(pred)

#     weight = torch.zeros(10)
#     for i in range(10):
#         if pred[i].item() > 0.5:
#             weight[i] = 0.0027
#         else:
#             weight[i] = 0.00783
    
#     total_loss = ideal_weights - weight
#     print(ideal_weights)
#     print(weight)
#     print(total_loss)

#     loss = loss_fn(pred, total_loss.float())
#     print(loss)
#     loss.backward()
#     optimizer.step()

In [17]:
# class customAutograd(torch.autograd.Function):
#     @staticmethod
#     def forward(ctx, input):
#         """
#         In the forward pass we receive a Tensor containing the input and return
#         a Tensor containing the output. ctx is a context object that can be used
#         to stash information for backward computation. You can cache arbitrary
#         objects for use in the backward pass using the ctx.save_for_backward method.
#         """
#         ctx.save_for_backward(input)
#         return input / 2

#     @staticmethod
#     def backward(ctx, grad_output):
#         """
#         In the backward pass we receive a Tensor containing the gradient of the loss
#         with respect to the output, and we need to compute the gradient of the loss
#         with respect to the input.
#         """
#         input, = ctx.saved_tensors
#         return grad_output * input

In [18]:
# class testNet(nn.Module):
#     def __init__(self):
#         super(testNet, self).__init__()
#         torch.manual_seed(12)
#         self.elem_material = nn.Parameter(torch.rand(10), requires_grad=True)
    
#     def forward(self):
#         print(self.elem_material)

#         E_vector = 131 / (1 + torch.exp(-10 * (self.elem_material - torch.tensor([0.5] * 10)))) + 69
#         Stif = E_vector * 10
#         u = Stif / 2

#         return u


In [19]:
# net = testNet()

# num_epochs = 10
# learning_rate = 1
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

# ideal_u = torch.tensor([5.0] * 10)

# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     u = net()

#     loss = loss_fn(u, ideal_u)
#     loss.backward()

#     for name, param in net.named_parameters():
#         if param.requires_grad:
#             print(name, param.grad)

#     optimizer.step()

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
class testFunc(torch.autograd.Function):
    @staticmethod
    def forward(ctx, mat):
        ctx.save_for_backward(mat)

        return torch.linalg.inv(mat)
    
    def backward(ctx, grad_output):
        mat = ctx.saved_tensors[0]
        print(mat)
        matKron = torch.tensor(torch.kron(mat, mat), requires_grad=True)
        matKron.backward(torch.ones_like(matKron))
    
        return torch.mm(grad_output, matKron)

In [22]:
testFunction = testFunc.apply

In [23]:
class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.mat = nn.Parameter(torch.rand((20, 20), requires_grad=True, device=device))

    def forward(self):    
        return testFunction(self.mat)

In [24]:
# net = testNet()
# net.to(device)
# optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
# loss_fn = nn.MSELoss()

# target = torch.zeros((20, 20), device=device)

# for epoch in range(10):
#     output = net()
#     loss = loss_fn(output, target)
#     loss.backward()
#     optimizer.step()

In [25]:
x = torch.rand((10, 10))
y = torch.linalg.inv(x)
z = torch.zeros((10, 10))
for i in range(10):
    for j in range(10):
        z[i][j] = y[i][j]
print(x.shape, x.stride(), '\n', x)
print(y.shape, y.stride(), '\n', y)
print(z.shape, z.stride(), '\n', z)

torch.Size([10, 10]) (10, 1) 
 tensor([[0.0271, 0.2151, 0.7820, 0.5134, 0.4355, 0.3072, 0.0737, 0.7715, 0.0422,
         0.2670],
        [0.5125, 0.3548, 0.1606, 0.1903, 0.1135, 0.8697, 0.9632, 0.8893, 0.5225,
         0.6941],
        [0.3968, 0.0639, 0.5050, 0.7526, 0.0627, 0.6014, 0.0559, 0.1256, 0.7948,
         0.4883],
        [0.2580, 0.5608, 0.7124, 0.1630, 0.7106, 0.1622, 0.3188, 0.5376, 0.8680,
         0.2969],
        [0.0148, 0.0853, 0.7770, 0.9735, 0.0798, 0.1825, 0.4132, 0.7279, 0.1326,
         0.1005],
        [0.1620, 0.2401, 0.6960, 0.5899, 0.8300, 0.1688, 0.3844, 0.9515, 0.6833,
         0.1707],
        [0.1260, 0.9503, 0.0543, 0.4494, 0.8870, 0.4177, 0.5973, 0.1598, 0.6201,
         0.9309],
        [0.6626, 0.9492, 0.9596, 0.8604, 0.9251, 0.8222, 0.7749, 0.1380, 0.8630,
         0.8848],
        [0.2015, 0.6912, 0.5956, 0.5532, 0.2850, 0.2663, 0.3537, 0.4955, 0.7461,
         0.0488],
        [0.8773, 0.7659, 0.6074, 0.7032, 0.8705, 0.1476, 0.5506, 0.8056, 0.617

In [26]:
mat1 = torch.tensor([1, 2, 3])
mat2 = torch.tensor([[3, 4, 5, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 3, 4, 5, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 3, 4, 5]])

mat1 @ mat2

tensor([ 3,  4,  5,  6,  8, 10,  9, 12, 15])

In [27]:
mat3 = torch.zeros((10, 100))
k = 0
for i in range(10):
    for j in range(10):
        mat3[i][j + k] = mat2[j]
    k = k + 10
mat3

RuntimeError: expand(torch.LongTensor{[9]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)

In [28]:
def manualKron(mat1, mat2):
    initSize = 2
    kronSize = initSize ** 2
    kron = torch.zeros((kronSize, kronSize))
    x = 0
    for i in range(initSize):
        for k in range(initSize):
            y = 0
            for j in range(initSize):
                for l in range(initSize):
                    kron[x][y] = mat1[i][j].item() * mat2[k][l].item()
                    y += 1
            x += 1
    return kron

In [34]:
mat1 = torch.tensor([[2.0, 2.0, 3.0, 0.0],
                     [2.0, 3.0, 4.0, 5.0],
                     [3.0, 4.0, 5.0, 6.0],
                     [0.0, 5.0, 6.0, 7.0]], requires_grad=True)
mat2 = torch.linalg.inv(mat1)
mat2

tensor([[ 0.0000, -0.2500,  0.5000, -0.2500],
        [-0.2500, -4.0625,  2.8750,  0.4375],
        [ 0.5000,  2.8750, -2.2500, -0.1250],
        [-0.2500,  0.4375, -0.1250, -0.0625]], grad_fn=<LinalgInvExBackward0>)